In [16]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf  # for gathering historical data
import talib as ta  # technical analysis library with indicators


In [17]:
# downloading apple stack data for 1 day interval of past 2 years
aapl = yf.download(tickers="aapl", interval="1d", period="2y",auto_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [18]:
# check the data
aapl.head()

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2023-02-22,147.423279,148.452889,145.690751,147.383670,51011300
2023-02-23,147.908371,148.838988,145.769948,148.591484,48394200
2023-02-24,145.245239,145.720443,144.265118,145.641240,55469600
2023-02-27,146.443146,147.680666,145.977837,146.235251,44998500
2023-02-28,145.938263,147.591588,145.364052,145.581857,50547000


"""
Above we can see that we have downloaded apple stock data.
by doing .head() we get first 5 samples of the data.
"""

In [19]:
# data info 
aapl.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 502 entries, 2023-02-22 to 2025-02-21
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, AAPL)   502 non-null    float64
 1   (High, AAPL)    502 non-null    float64
 2   (Low, AAPL)     502 non-null    float64
 3   (Open, AAPL)    502 non-null    float64
 4   (Volume, AAPL)  502 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 23.5 KB


""" Above we can data info of last two years
no. of rows 500 and columns ( Open High Low Close Volume) or (OHLCV) data format
and their is no Null value which means data is consistant as a Time Serise data.
"""


In [20]:
# describe the data
aapl.describe()

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
count,502.000000,502.000000,502.000000,502.000000,5.020000e+02
mean,195.291173,196.859915,193.469475,195.051866,5.681398e+07
std,27.249692,27.491494,26.851064,27.183730,2.476890e+07
min,143.859222,145.245263,142.463305,142.938524,2.323470e+07
25%,173.672455,175.570692,172.279632,173.527515,4.391898e+07
50%,188.259712,189.387939,186.988465,188.202327,5.148065e+07
75%,222.506615,225.141998,220.891645,223.578359,6.259602e+07
max,258.735504,259.814335,257.347047,257.906429,3.186799e+08


"""
Now above we have describe the data where IQR can be seen along with mean median mode and sample count and also standard deviation"""

In [21]:
# converting the data info pandas data frame and reindexing the headers or columns name
data = pd.DataFrame({
    'Date': aapl.index, 
    'Open': aapl['Open'].values.flatten(),  
    'High': aapl['High'].values.flatten(), 
    'Low': aapl['Low'].values.flatten(),    
    'Close': aapl['Close'].values.flatten()
})

In [22]:
# now calculate the RSI
data["rsi"] = ta.RSI(data["Close"], 14)

In [23]:
# Print data frame
data.tail()

,Date,Open,High,Low,Close,rsi
497,2025-02-14,241.250000,245.550003,240.990005,244.600006,60.633526
498,2025-02-18,244.149994,245.179993,241.839996,244.470001,60.452968
499,2025-02-19,244.660004,246.009995,243.160004,244.869995,60.839364
500,2025-02-20,244.940002,246.779999,244.289993,245.830002,61.803953
501,2025-02-21,245.949997,248.690002,245.220001,245.550003,61.329462


""" Above you can see that last five rows of data with .tail() function has been printed.
Here we can se that at the window interval 14 we have calculated RSI """

In [24]:
# simple rsi strategy or set of rules to generate buy and sell signals
upper_band, lower_band = 70, 30

# Initialize the Signal column with None
data["Signal"] = None

# # Use vectorized operations to set the Signal column
# data.loc[data["rsi"] > upper_band, "Signal"] = "Sell"
# data.loc[data["rsi"] < lower_band, "Signal"] = "Buy"


last_action = None

for index, row in data.iterrows():
    if row["rsi"] < lower_band and last_action != "Buy":
        # First buy action
        data.at[index, "Signal"] = "Buy"
        last_action = "Buy"  # Update last action to 'Buy'
    elif row["rsi"] > upper_band and last_action == "Buy":
        # Sell action after a buy
        data.at[index, "Signal"] = "Sell"
        last_action = "Sell"  # Update last action to 'Sell'
    elif row["rsi"] < lower_band and last_action == "Sell":
        # Buy action after a sell
        data.at[index, "Signal"] = "Buy"
        last_action = "Buy"  # Update last action to 'Buy'


In [25]:
# Buy and sell signals
buy_signals = data[data["Signal"] == "Buy"]
sell_signals = data[data["Signal"] == "Sell"]

In [26]:
# buy signal data
buy_signals.head()

,Date,Open,High,Low,Close,rsi,Signal
114,2023-08-07,180.561008,181.552393,175.822187,177.309265,29.566267,Buy
218,2024-01-04,181.062914,181.997307,179.800504,180.824356,29.828340,Buy
479,2025-01-21,223.753984,224.173521,219.139063,222.395477,29.697464,Buy


In [27]:
# sell signal data
sell_signals.head()


,Date,Open,High,Low,Close,rsi,Signal
186,2023-11-16,188.438641,189.820344,187.524118,188.577805,71.238865,Sell
309,2024-05-15,187.280568,190.011381,186.742369,189.084503,71.282569,Sell


#### Now will plot the buy and sell signal data with help of matplotlib library

In [28]:
plt.figure(figsize=(10, 5))
plt.plot(data['Date'], data['rsi'], label='RSI', color='blue', linestyle='-')
plt.axhline(y=upper_band, color='red', linestyle='--', label='Upper Band (70)')
plt.axhline(y=lower_band, color='green', linestyle='--', label='Lower Band (30)')

# Mark Buy signals
plt.scatter(buy_signals['Date'], buy_signals['rsi'], color='green', marker='^', s=100, label='Buy Signal')

# Mark Sell signals
plt.scatter(sell_signals['Date'], sell_signals['rsi'], color='red', marker='v', s=100, label='Sell Signal')

# Adding titles and labels
plt.title('RSI with Buy and Sell Signals')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.xticks(rotation=45)
plt.legend()
plt.grid()

# Show the plot
plt.tight_layout()
plt.show()


,Date,Open,High,Low,Close,rsi,Signal
0,2023-02-22,147.383670,148.452889,145.690751,147.423279,NaN,None
1,2023-02-23,148.591484,148.838988,145.769948,147.908371,NaN,None
2,2023-02-24,145.641240,145.720443,144.265118,145.245239,NaN,None
3,2023-02-27,146.235251,147.680666,145.977837,146.443146,NaN,None
4,2023-02-28,145.581857,147.591588,145.364052,145.938263,NaN,None
...,...,...,...,...,...,...,...
497,2025-02-14,241.250000,245.550003,240.990005,244.600006,60.633526,None
498,2025-02-18,244.149994,245.179993,241.839996,244.470001,60.452968,None
499,2025-02-19,244.660004,246.009995,243.160004,244.869995,60.839364,None
500,2025-02-20,244.940002,246.779999,244.289993,245.830002,61.803953,None
